# Model 1: Modeling Selected Actions

The purpose of this model is to predict the action that the bot will select for a given state of the simulation.

## Preliminary Setup

### Imports

In [ ]:
import os
import numpy as np
import random
import math
import json
import pandas as pd
import ast
import datetime
import pickle

In [ ]:
# %pip install json

In [ ]:
import nbimporter

### Additional Bot Config

In [ ]:
import Bot1

In [ ]:
file_name = 'Data/Model1/model1_data_raw.csv'

In [ ]:
%run Bot1.ipynb

In [ ]:
grid, open_cells = create_grid() # Fixed grid orientation

In [ ]:
def determine_probabilities(bot, matrix):
    directions = {'up': (bot[0], bot[1] - 1), 
                  'down': (bot[0], bot[1] + 1), 
                  'left': (bot[0] - 1, bot[1]), 
                  'right': (bot[0] + 1, bot[1]),
                  'stay': bot}
    return [matrix.get(directions[direction], 0) for direction in ['up', 'down', 'left', 'right', 'stay']]

In [ ]:
def determine_d_crew(ship, bot, alpha, d_lookup_table, crew_list, crew_matrix, open_cells):
    directions = {'up': (bot[0], bot[1] - 1), 
                  'down': (bot[0], bot[1] + 1), 
                  'left': (bot[0] - 1, bot[1]), 
                  'right': (bot[0] + 1, bot[1]),
                  'stay': bot}
    
    direction_values = set(directions.values())
    
    filtered_crew_matrix = {k: v for k, v in crew_matrix.items() if k not in direction_values}
    
    if filtered_crew_matrix:
        max_crew_cell = max(filtered_crew_matrix, key=filtered_crew_matrix.get)
    else:
        return [0] * 5, d_lookup_table
        
    d_list = []
    
    for direction in ['up', 'down', 'left', 'right', 'stay']:
        if (directions[direction] in open_cells or directions[direction] == bot) and directions[direction] not in crew_list:
            _, d_lookup_table = crew_sensor(ship, directions[direction], alpha, d_lookup_table, crew_list)
            d_dict = d_lookup_table.get(directions[direction])
            d_list.append(1 / d_dict[max_crew_cell[0], max_crew_cell[1]])
        else:
            d_list.append(0)
    
    return d_list, d_lookup_table

In [ ]:
def determine_d_alien(ship, bot, alpha, d_lookup_table, alien_list, crew_list, alien_matrix, open_cells):
    directions = {'up': (bot[0], bot[1] - 1), 
                  'down': (bot[0], bot[1] + 1), 
                  'left': (bot[0] - 1, bot[1]), 
                  'right': (bot[0] + 1, bot[1]),
                  'stay': bot}
    
    direction_values = set(directions.values())
    
    filtered_alien_matrix = {k: v for k, v in alien_matrix.items() if k not in direction_values}
    
    if filtered_alien_matrix:
        max_alien_cell = max(filtered_alien_matrix, key=filtered_alien_matrix.get)
    else:
        return [0] * 5, d_lookup_table
        
    d_list = []
    
    for direction in ['up', 'down', 'left', 'right', 'stay']:
        if (directions[direction] in open_cells or directions[direction] == bot) and directions[direction] not in crew_list and directions[direction] not in alien_list:
            _, d_lookup_table = crew_sensor(ship, directions[direction], alpha, d_lookup_table, crew_list)
            d_dict = d_lookup_table.get(directions[direction])
            d_list.append(1 / d_dict[max_alien_cell[0], max_alien_cell[1]])
        else:
            d_list.append(0)
    
    return d_list, d_lookup_table

In [ ]:
def Bot1_collect_data(k, alpha, max_iter, timeout):
    global grid, open_cells
    
    grid, open_cells = reset_grid(grid, open_cells)
    bot, ship, open_cells = place_bot(grid, open_cells)

    crew_list = []
    alien_list = []
    d_lookup_table = {}
    
    data_log = [] # Data Log Initialization

    crew_list, ship = place_crew(ship, open_cells, crew_list)
    alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

    alien_matrix = initialize_alienmatrix(open_cells, bot, k)
    crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
    
    alien_detected = alien_sensor(alien_list, bot, k) # Initially Run Alien Sensor
    crew_detected, d_lookup_table = crew_sensor(ship, bot, alpha, d_lookup_table, crew_list) # Initially Run Crew Sensor
    
    next_move_str = 'stay'

    win_count = 0
    loss_count = 0
    move = 0
    win_move_count = []
    marker = 0

    while (win_count + loss_count) < max_iter:
        neighbors = check_valid_neighbors(len(ship), bot[0], bot[1])
        open_moves = [neigh for neigh in neighbors if (grid[neigh] != 1)]
        open_moves.append(bot) # Bot can stay in place 
        next_move = determine_move(open_moves, alien_matrix, crew_matrix)
        
#         alien_matrix_str_keys = {str(key): round(value, 5) for key, value in alien_matrix.items()}
#         crew_matrix_str_keys = {str(key): round(value, 5) for key, value in crew_matrix.items()}

#         alien_matrix_json = json.dumps(alien_matrix_str_keys)
#         crew_matrix_json = json.dumps(crew_matrix_str_keys)

#         alien_matrix_flat = [round(alien_matrix.get((x, y), 0), 5) for x in range(30) for y in range(30)]
#         crew_matrix_flat = [round(crew_matrix.get((x, y), 0), 5) for x in range(30) for y in range(30)]

        alien_probs = determine_probabilities(bot, alien_matrix)
        crew_probs = determine_probabilities(bot, crew_matrix)
        d_crew, d_lookup_table = determine_d_crew(ship, bot, alpha, d_lookup_table, crew_list, crew_matrix, open_cells) # Find shortest distance from highest probability crew cell to all neighbors
        d_alien, d_lookup_table = determine_d_alien(ship, bot, alpha, d_lookup_table, alien_list, crew_list, alien_matrix, open_cells) # Find shortest distance from highest probability alien cell to all neighbors
        
        # Convert relative move to string      
        if next_move[0] > bot[0]:
            next_move_str = 'right'
        elif next_move[0] < bot[0]:
            next_move_str = 'left'
        elif next_move[1] > bot[1]:
            next_move_str = 'up'
        elif next_move[1] < bot[1]:
            next_move_str = 'down'
        else:
            next_move_str = 'stay'
        
        # One-Hot Encoding
        actions = {'up': [1, 0, 0, 0, 0], 'down': [0, 1, 0, 0, 0], 'left': [0, 0, 1, 0, 0], 'right': [0, 0, 0, 1, 0], 'stay': [0, 0, 0, 0, 1]}
        best_move_encoded = actions[next_move_str]
        
        log_entry = {
            'bot_x': bot[0],
            'bot_y': bot[1],
            
            'alien_up': alien_probs[0],
            'alien_down': alien_probs[1],
            'alien_left': alien_probs[2],
            'alien_right': alien_probs[3],
            'alien_stay': alien_probs[4],
            
            'crew_up': crew_probs[0],
            'crew_down': crew_probs[1],
            'crew_left': crew_probs[2],
            'crew_right': crew_probs[3],
            
            'd_crew_up': np.float32(d_crew[0]),
            'd_crew_down': np.float32(d_crew[1]),
            'd_crew_left': np.float32(d_crew[2]),
            'd_crew_right': np.float32(d_crew[3]),
            'd_crew_stay': np.float32(d_crew[4]),
            
            'd_alien_up': np.float32(d_alien[0]),
            'd_alien_down': np.float32(d_alien[1]),
            'd_alien_left': np.float32(d_alien[2]),
            'd_alien_right': np.float32(d_alien[3]),
            'd_alien_stay': np.float32(d_alien[4]),
            
            'alien_detected': 1 if alien_detected else 0,
            'crew_detected': 1 if crew_detected else 0,
            
            'chosen_action': best_move_encoded
        }
        data_log.append(log_entry)
        
#         log_entry = {
#             'bot_x': bot[0],
#             'bot_y': bot[1],
            
#             'alien_up': alien_probs[0],
#             'alien_down': alien_probs[1],
#             'alien_left': alien_probs[2],
#             'alien_right': alien_probs[3],
#             'alien_stay': alien_probs[4],
            
#             'crew_up': crew_probs[0],
#             'crew_down': crew_probs[1],
#             'crew_left': crew_probs[2],
#             'crew_right': crew_probs[3],
            
#             'd_crew_up': np.float32(d_crew[0]),
#             'd_crew_down': np.float32(d_crew[1]),
#             'd_crew_left': np.float32(d_crew[2]),
#             'd_crew_right': np.float32(d_crew[3]),
#             'd_crew_stay': np.float32(d_crew[4]),
            
#             'alien_detected': 1 if alien_detected else 0,
#             'crew_detected': 1 if crew_detected else 0,
            
#             'chosen_action': best_move_encoded
#         }
#         data_log.append(log_entry)

#         log_entry = {
#             'bot_x': bot[0],
#             'bot_y': bot[1],
            
#             **{'alien_' + str(key): np.float32(value) for key, value in alien_matrix.items()},
#             **{'crew_' + str(key): np.float32(value) for key, value in crew_matrix.items()},
            
#             'alien_detected': 1 if alien_detected else 0,
#             'crew_detected': 1 if crew_detected else 0,
            
#             'chosen_action': best_move_encoded
#         }
#         data_log.append(log_entry)
        
        prev_win_count = win_count
        bot, crew_list, ship, open_cells, win_count, marker = move_bot(ship, bot, next_move, crew_list, alien_list, open_cells, win_count, 1)
        move += 1

        if marker == 1 or move >= timeout:
            loss_count += 1
            print(f"Bot captured! Win Count: {win_count}, Loss Count: {loss_count}")

            grid, open_cells = reset_grid(grid, open_cells)
            bot, ship, open_cells = place_bot(grid, open_cells)
            crew_list = []
            alien_list = []
            d_lookup_table = {}

            crew_list, ship = place_crew(ship, open_cells, crew_list)
            alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

            alien_matrix = initialize_alienmatrix(open_cells, bot, k)
            crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
            marker = 0
            move = 0

            continue

        if win_count > prev_win_count:
            print(f"Crew saved! Win Count: {win_count}, Loss Count: {loss_count}")
            win_move_count.append(move)
            move = 0
            d_lookup_table = {}
            alien_matrix = initialize_alienmatrix(open_cells, bot, k)
            crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
        
        print(f"Bot: {bot}, Crew: {crew_list}, Aliens: {alien_list}")

        alien_matrix, crew_matrix = update_afterbotmove(bot, alien_matrix, crew_matrix)
        
        marker, alien_list, ship = move_aliens(ship, alien_list, bot) # Move alien randomly

        if marker == 1 or move >= timeout:
            loss_count += 1
            print(f"Bot captured! Win Count: {win_count}, Loss Count: {loss_count}")

            grid, open_cells = reset_grid(grid, open_cells)
            bot, ship, open_cells = place_bot(grid, open_cells)
            crew_list = []
            alien_list = []
            d_lookup_table = {}

            crew_list, ship = place_crew(ship, open_cells, crew_list)
            alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

            alien_matrix = initialize_alienmatrix(open_cells, bot, k)
            crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
            marker = 0
            move = 0

            continue
        
        alien_matrix = update_afteralienmove(ship, alien_list, alien_matrix) # Update after alien move
        
        alien_detected = alien_sensor(alien_list, bot, k) # Run Alien Sensor
        crew_detected, d_lookup_table = crew_sensor(ship, bot, alpha, d_lookup_table, crew_list) # Run Crew Sensor
        
        alien_matrix = update_alienmatrix(alien_matrix, alien_detected, bot, k) # Update based on alien sensor

        crew_matrix = update_crewmatrix(crew_matrix, crew_detected, d_lookup_table, bot, alpha) # Update based on crew sensor
    
    df = pd.DataFrame(data_log)
    
    if os.path.isfile(file_name):
        df.to_csv(file_name, mode='a', index=False, header=False)
    else:
        df.to_csv(file_name, mode='w', index=False, header=True)

    return sum(win_move_count) // max(1, len(win_move_count)), (win_count / max(1, (win_count + loss_count))), win_count

In [ ]:
def Bot1_simulation(alpha_values, k_values, max_iter, timeout, num_simulations):
    avg_rescue_moves = {k: [] for k in k_values}
    prob_crew_rescue = {k: [] for k in k_values}
    avg_crew_saved = {k: [] for k in k_values}

    for k in k_values:
        for alpha in alpha_values:
            total_metric1, total_metric2, total_metric3 = 0, 0, 0
            
            for i in range(num_simulations):
                metric1, metric2, metric3 = Bot1_collect_data(k, alpha, max_iter, timeout)
                total_metric1 += metric1
                total_metric2 += metric2
                total_metric3 += metric3

            avg_metric1 = total_metric1 / num_simulations
            avg_metric2 = total_metric2 / num_simulations
            avg_metric3 = total_metric3 / num_simulations

            print(f"k: {k}, Alpha: {alpha}\nAverage Rescue Moves: {avg_metric1}\nProbability of Crew Rescue: {avg_metric2}\nAverage Crew Saved: {avg_metric3}\n")

            avg_rescue_moves[k].append(avg_metric1)
            prob_crew_rescue[k].append(avg_metric2)
            avg_crew_saved[k].append(avg_metric3)

    return avg_rescue_moves, prob_crew_rescue, avg_crew_saved

In [ ]:
def one_alien_one_crew(alpha_values, k_values, max_iter, timeout, num_simulations):
    bot1_avg_rescue_moves, bot1_prob_crew_rescue, bot1_avg_crew_saved = Bot1_simulation(alpha_values, k_values, max_iter, timeout, num_simulations)

    bot1_prob_crew_rescue = {k: [round(prob, 3) for prob in probs] for k, probs in bot1_prob_crew_rescue.items()}

    print(bot1_avg_rescue_moves, bot1_prob_crew_rescue, bot1_avg_crew_saved, "\n")

In [ ]:
alpha_values = [0.004]
k_values = [3]
max_iter = 30
timeout = 10000
num_simulations = 100

In [ ]:
# one_alien_one_crew(alpha_values, k_values, max_iter, timeout, num_simulations)

In [ ]:
# k: 3, Alpha: 0.004
# Average Rescue Moves: 585.05
# Probability of Crew Rescue: 0.71
# Average Crew Saved: 21.3

# {3: [585.05]} {3: [0.71]} {3: [21.3]} 

### Model 1 Training Functions

In [ ]:
# Softmax Function
def softmax(z):
    e_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return e_z / np.sum(e_z, axis=1, keepdims=True)

In [ ]:
# Initialize weights and biases
def init_params(num_features, num_classes):
    W = np.random.randn(num_features, num_classes) * 0.01 # Initialize to a small random number
    b = np.zeros((1, num_classes))
#     print(W, b, W.shape, b.shape)
    return W, b

In [ ]:
# Loss Function
def loss_function(y_true, y_pred, valid):
    n = y_true.shape[0]

    valid_y_pred = y_pred * valid
    valid_y_pred_sum = valid_y_pred.sum(axis=1, keepdims=True)
    valid_y_pred /= valid_y_pred_sum
    
    y_true_array = np.array(y_true['chosen_action'].apply(ast.literal_eval).tolist())
#     print(y_true_array, y_true_array.shape, valid_y_pred, valid_y_pred.shape)

    loss = -np.sum(y_true_array * np.log(valid_y_pred + 1e-15)) / n  # Add a small number to prevent log(0)
    return loss

In [ ]:
# Calculate the gradient
def compute_gradient(X, y_true, y_pred, valid):
    n = X.shape[0]
    
    valid_y_pred = y_pred * valid
    valid_y_pred_sum = valid_y_pred.sum(axis=1, keepdims=True)
    valid_y_pred /= valid_y_pred_sum
    
    y_true_array = np.array(y_true['chosen_action'].apply(ast.literal_eval).tolist())
    
    dW = (1 / n) * np.dot(X.T, (valid_y_pred - y_true_array))
    db = (1 / n) * np.sum(valid_y_pred - y_true_array, axis=0, keepdims=True)
    return dW, db

In [ ]:
# Prediction Function
def predict(X, W, b):
    z = np.dot(X, W) + b
    y_pred = softmax(z)
    return y_pred

In [ ]:
# Prediction Function to account for cases with invalid predictions
# For example, [0, 0.1, 0.2, 0.4, 0.3] would become [0, 0.1, 0, 0, 0.3] if left and right were invalid, and then normalization would cause it to become [0, 0.25, 0, 0, 0.75]
def predict_constrained(X, W, b, valid):
    y_pred = predict(X, W, b)
    
    valid_y_pred = y_pred * valid
    valid_y_pred_sum = valid_y_pred.sum(axis=1, keepdims=True)
    valid_y_pred /= valid_y_pred_sum
    
    one_hot_pred = np.zeros_like(valid_y_pred, dtype=int)
    one_hot_pred[np.arange(len(valid_y_pred)), np.argmax(valid_y_pred, axis=1)] = 1

    return one_hot_pred

In [ ]:
# Prediction Function to account for cases with invalid predictions (with a stochastic component)
def predict_constrained_stochastic(X, W, b, valid):
    y_pred = predict(X, W, b)
    
    valid_y_pred = y_pred * valid
    valid_y_pred_sum = valid_y_pred.sum(axis=1, keepdims=True)
    valid_y_pred /= valid_y_pred_sum
    
    one_hot_pred = np.zeros_like(valid_y_pred, dtype=int)
    
    random_choice = random.randint(1, 5)
    
    if random_choice == 1:
        non_zero_cols = np.nonzero(valid_y_pred[0])[0]
        col = np.random.choice(non_zero_cols)
        one_hot_pred[0, col] = 1
    else:
        one_hot_pred[np.arange(len(valid_y_pred)), np.argmax(valid_y_pred, axis=1)] = 1

    return one_hot_pred

In [ ]:
# Train Function that uses only GD
def train(X, y, valid, alpha, epochs):
    num_features = X.shape[1]
    num_classes = 5
    n = X.shape[0]

    W, b = init_params(num_features, num_classes)

    previous_loss = float('inf')
    loss_list = []

    for epoch in range(epochs):
        current_loss = 0

        X_batch = X
        y_batch = y
        valid_batch = valid

        y_pred = predict(X_batch, W, b)
        batch_loss = loss_function(y_batch, y_pred, valid_batch)
        current_loss += batch_loss

        dW, db = compute_gradient(X_batch, y_batch, y_pred, valid_batch)
        W -= alpha * dW
        b -= alpha * db
        
        current_loss /= n
        loss_list.append(current_loss)

        previous_loss = current_loss

        print(f"Epoch {epoch}, Loss: {current_loss}")

    return W, b, loss_list

In [ ]:
def is_valid(x, y, move, grid, open_cells):
    if move == 'up' and (x, y + 1) in open_cells:
        return 1
    elif move == 'down' and (x, y - 1) in open_cells:
        return 1
    elif move == 'left' and (x - 1, y) in open_cells:
        return 1
    elif move == 'right' and (x + 1, y) in open_cells:
        return 1
    elif move == 'stay':
        return 1
    else:
        return 0

In [ ]:
grid, open_cells = reset_grid(grid, open_cells)

def create_valid_matrix(X):
    global grid, open_cells
    directions = ['up', 'down', 'left', 'right', 'stay']
    valid_list = []
    for i in range(len(X)):
        x, y = X.iloc[i, 0], X.iloc[i, 1]
        validity_for_each_direction = [is_valid(x, y, move, grid, open_cells) for move in directions]
        valid_list.append(validity_for_each_direction)

    valid_array = np.array(valid_list)
    return valid_array

In [ ]:
def plot_training_loss(loss_list):
    directory = "Results/Model1/Plots"
    
    if not os.path.exists(directory):
        os.makedirs(directory)

    filename = f"loss_plot_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.png"
    filepath = os.path.join(directory, filename)
    
    plt.figure(figsize=(10, 6))
    plt.plot(loss_list, label='Loss per Epoch')
    plt.title('Model Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.savefig(filepath)
    plt.show()

### Model 1 Testing Functions

In [ ]:
def calculate_accuracy(y_true, y_pred):
    correct_predictions = (y_true == y_pred)
    accuracy = correct_predictions.sum() / correct_predictions.size
    return accuracy

In [ ]:
def test(W, b, X_train, y_train, valid_train, X_test, y_test, valid_test): 
    y_train_df = y_train.squeeze()
    y_test_df = y_test.squeeze()

    y_train_true = np.array(y_train_df.apply(ast.literal_eval).tolist())
    y_test_true = np.array(y_test_df.apply(ast.literal_eval).tolist())
    
    y_train_pred = predict_constrained(X_train, W, b, valid_train)
    
#     print(y_train_true, y_train_pred)
    train_acc = calculate_accuracy(y_train_true, y_train_pred)

    y_test_pred = predict_constrained(X_test, W, b, valid_test)
    test_acc = calculate_accuracy(y_test_true, y_test_pred)
    
#     print(f"Training Accuracy: {train_acc}\nTesting Accuracy: {test_acc}")

    return train_acc, test_acc

### Principle Component Analysis

In [ ]:
def standardize_data(X):
    X_standardized = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
    return X_standardized

In [ ]:
def compute_and_sort_eigens(X):
    cov_matrix = np.cov(X.T)
    cov_matrix = np.real(cov_matrix)
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

    idx = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[idx]
    eigenvectors = eigenvectors[:, idx]
    
    return eigenvalues, eigenvectors

In [ ]:
def filter_by_eigenvectors(X, eigenvectors, k):
    eigenvectors = eigenvectors[:, :k]
    return np.dot(X, eigenvectors)

In [ ]:
def pca(X, k):
    X_standardized = standardize_data(X)
    eigenvalues, eigenvectors = compute_and_sort_eigens(X_standardized)
    X_transformed = filter_by_eigenvectors(X_standardized, eigenvectors, k)
    
#     X_standardized = standardize_data(X)
#     U, s, Vt = np.linalg.svd(X_standardized, full_matrices=False)
#     components = Vt[:k]
#     X_transformed = np.dot(X_standardized, components.T)
    
    return X_transformed

### Data Preprocessing

#### data

In [ ]:
# data = pd.read_csv('Data/Model1/Final/model1_data_raw.csv')

In [ ]:
data = pd.read_csv('Data/Model1/model1_data_raw.csv')

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
# Obsolete

# chunksize = 10000
# chunks = pd.read_csv('Data/Model1/model1_data_raw.csv', chunksize=chunksize)

# for i, chunk in enumerate(chunks):
#     for col in chunk.select_dtypes(include=['float64']).columns:
#         chunk[col] = chunk[col].astype('float32')
    
#     # Append each chunk to a new CSV
#     if i == 0:
#         chunk.to_csv('Data/Model1/model1_data_raw_lite.csv', index=False, mode='w')
#     else:
#         chunk.to_csv('Data/Model1/model1_data_raw_lite.csv', index=False, mode='a', header=False)

In [ ]:
# Obsolete

# data['alien_prob'] = data[['alien_up', 'alien_down', 'alien_left', 'alien_right', 'alien_stay']].values.tolist()
# data['crew_prob'] = data[['crew_up', 'crew_down', 'crew_left', 'crew_right', 'crew_stay']].values.tolist()

In [ ]:
# Obsolete

# data = data.drop(columns=['alien_up', 'alien_down', 'alien_left', 'alien_right', 'alien_stay', 'crew_up', 'crew_down', 'crew_left', 'crew_right', 'crew_stay'], axis=1)

In [ ]:
# Obsolete

# output = data.pop('chosen_action')
# data.insert(len(data.columns), output.name, output)

In [ ]:
# data.head()

In [ ]:
model1_df = data.drop_duplicates()
model1_df.shape

In [ ]:
# Obsolete

# crew_columns = ['crew_up', 'crew_down', 'crew_left', 'crew_right', 'crew_stay']
# crew_sum = model1_df[crew_columns].sum(axis=1)

# model1_df[crew_columns] = np.where(crew_sum[:, None] != 0, model1_df[crew_columns].div(crew_sum, axis=0), model1_df[crew_columns])

In [ ]:
# Obsolete

# for col in model1_df.columns:
#     if model1_df[col].apply(lambda x: isinstance(x, tuple)).any():
#         model1_df[col] = model1_df[col].apply(list)

#### model1_df

In [ ]:
# model1_df = pd.read_csv('Data/Model1/Final/model1_data.csv')

In [ ]:
model1_df.head()

In [ ]:
model1_df.shape

In [ ]:
# model1_df = model1_df.drop(columns=['crew_stay'])
# model1_df

In [ ]:
crew_columns = ['crew_up', 'crew_down', 'crew_left', 'crew_right']
model1_df['sum_crew'] = model1_df[crew_columns].sum(axis=1)

for col in crew_columns:
    model1_df.loc[model1_df['sum_crew'] != 0, col] = model1_df[col] / model1_df['sum_crew']

model1_df.drop('sum_crew', axis=1, inplace=True)

model1_df

In [ ]:
class_labels = model1_df['chosen_action'].apply(ast.literal_eval)
class_counts = class_labels.value_counts()
class_counts

In [ ]:
model1_df.to_csv('Data/Model1/model1_data.csv', index=False)

In [ ]:
X_df = model1_df.iloc[:,:-1]
y_df = model1_df.iloc[:,-1:]

In [ ]:
# Obsolete

# k_components = 100
# X_slice = X_df.iloc[:10000]
# X_np = pca(X_slice.values, k_components)
# X_pca = pd.DataFrame(X_np, columns=[f'PC{i + 1}' for i in range(k_components)])
# X_pca

In [ ]:
# Obsolete

# Remove columns with variance lower than threshold
# threshold = 0.000001

# variances = X_df.var()

# high_variance_features = variances[variances > threshold].index.tolist()

# X_pca = X_df[high_variance_features]
# X_pca

In [ ]:
train_size = int(0.8 * len(model1_df))

train_df = model1_df[:train_size]
test_df = model1_df[train_size:]

#### train_df + test_df

In [ ]:
# train_df = pd.read_csv('Data/Model1/Final/model1_train.csv')
# test_df = pd.read_csv('Data/Model1/Final/model1_test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
test_df.head()

In [ ]:
test_df.shape

In [ ]:
class_labels = train_df['chosen_action'].apply(ast.literal_eval)
class_counts = class_labels.value_counts()
class_counts

In [ ]:
class_labels = test_df['chosen_action'].apply(ast.literal_eval)
class_counts = class_labels.value_counts()
class_counts

In [ ]:
# class_labels = train_df['bot_x']
# class_labels = class_labels.sort_values(ascending=True)
# sorted_unique_labels = class_labels.unique()
# class_counts = class_labels.value_counts()
# class_counts = class_counts.reindex(sorted_unique_labels)
# class_counts

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
train_df.to_csv('Data/Model1/model1_train.csv', index=False)
test_df.to_csv('Data/Model1/model1_test.csv', index=False)

#### X_train_df + y_train_df

In [ ]:
X_train_df = train_df.iloc[:,:-1]
y_train_df = train_df.iloc[:,-1:]

X_test_df = test_df.iloc[:,:-1]
y_test_df = test_df.iloc[:,-1:]

In [ ]:
# X_train_df = pd.read_csv('Data/Model1/Final/X_train.csv')
# y_train_df = pd.read_csv('Data/Model1/Final/y_train.csv')

# X_test_df = pd.read_csv('Data/Model1/Final/X_test.csv')
# y_test_df = pd.read_csv('Data/Model1/Final/y_test.csv')

In [ ]:
X_train_df.head()

In [ ]:
y_train_df.head()

In [ ]:
X_train_df.to_csv('Data/Model1/X_train.csv', index=False)
y_train_df.to_csv('Data/Model1/y_train.csv', index=False)

X_test_df.to_csv('Data/Model1/X_test.csv', index=False)
y_test_df.to_csv('Data/Model1/y_test.csv', index=False)

In [ ]:
# Obsolete

# size = int(0.8 * len(X_pca))

# X_train_df = X_pca[:size]
# y_train_df = y_df[:size]

# X_test_df = X_pca[size:]
# y_test_df = y_df[size:]

#### valid_train + valid_test

In [ ]:
valid_train = create_valid_matrix(X_train_df)
valid_test = create_valid_matrix(X_test_df)

In [ ]:
valid_train_df = pd.DataFrame(valid_train)
valid_test_df = pd.DataFrame(valid_test)

In [ ]:
valid_train_df.to_csv('Data/Model1/valid_train.csv', index=False)
valid_test_df.to_csv('Data/Model1/valid_test.csv', index=False)

#### valid_train_df + valid_test_df

In [ ]:
# valid_train_df = pd.read_csv('Data/Model1/Final/valid_train.csv')
# valid_test_df = pd.read_csv('Data/Model1/Final/valid_test.csv')

In [ ]:
# valid_train = valid_train_df.to_numpy()
# valid_test = valid_test_df.to_numpy()

In [ ]:
# valid_train_df.head()

In [ ]:
valid_train.shape

In [ ]:
valid_test.shape

## Model Training

In [ ]:
alpha = 0.01
epochs = 50
# initial_batch_size = 32
# loss_threshold = 0.0001

In [ ]:
# X_train_small = X_train[:70000]
# y_train_small = y_train[:70000]
# valid_train_small = valid_train[:70000]

# X_train_small.shape
# y_train_small.shape

# W, b, loss_list = train(X_train_small, y_train_small, valid_train_small, alpha, epochs)

In [ ]:
W, b, loss_list = train(X_train_df, y_train_df, valid_train, alpha, epochs)

In [ ]:
# Epoch 0, Loss: 1.6595809304718215e-05
# Epoch 1, Loss: 1.63714578251211e-05
# Epoch 2, Loss: 1.6352849397278687e-05
# Epoch 3, Loss: 1.634328202570606e-05
# Epoch 4, Loss: 1.6337172720916582e-05
# Epoch 5, Loss: 1.6332875447510344e-05
# Epoch 6, Loss: 1.6329641911552317e-05
# Epoch 7, Loss: 1.6327086305863702e-05
# Epoch 8, Loss: 1.6324988821639317e-05
# Epoch 9, Loss: 1.632321449309332e-05
# Epoch 10, Loss: 1.6321675689508683e-05
# Epoch 11, Loss: 1.6320313029994483e-05
# Epoch 12, Loss: 1.631908490015047e-05
# Epoch 13, Loss: 1.6317961333201676e-05
# Epoch 14, Loss: 1.631692025960384e-05
# Epoch 15, Loss: 1.6315945114580123e-05
# Epoch 16, Loss: 1.6315023260503623e-05
# Epoch 17, Loss: 1.6314144917283797e-05
# Epoch 18, Loss: 1.6313302419870933e-05
# Epoch 19, Loss: 1.6312489692297612e-05
# Epoch 20, Loss: 1.6311701868500904e-05
# Epoch 21, Loss: 1.6310935014710512e-05
# Epoch 22, Loss: 1.631018592339068e-05
# Epoch 23, Loss: 1.630945195839769e-05
# Epoch 24, Loss: 1.6308730937315134e-05
# Epoch 25, Loss: 1.6308021041120794e-05
# Epoch 26, Loss: 1.6307320744176402e-05
# Epoch 27, Loss: 1.6306628759488407e-05
# Epoch 28, Loss: 1.630594399555428e-05
# Epoch 29, Loss: 1.630526552207669e-05
# Epoch 30, Loss: 1.6304592542522774e-05
# Epoch 31, Loss: 1.630392437200849e-05
# Epoch 32, Loss: 1.6303260419356377e-05
# Epoch 33, Loss: 1.630260017244756e-05
# Epoch 34, Loss: 1.6301943186190942e-05
# Epoch 35, Loss: 1.6301289072586274e-05
# Epoch 36, Loss: 1.630063749247096e-05
# Epoch 37, Loss: 1.629998814863093e-05
# Epoch 38, Loss: 1.6299340780020882e-05
# Epoch 39, Loss: 1.629869515689333e-05
# Epoch 40, Loss: 1.6298051076674684e-05
# Epoch 41, Loss: 1.6297408360459563e-05
# Epoch 42, Loss: 1.6296766850018342e-05
# Epoch 43, Loss: 1.629612640523327e-05
# Epoch 44, Loss: 1.6295486901894094e-05
# Epoch 45, Loss: 1.6294848229795776e-05
# Epoch 46, Loss: 1.6294210291092177e-05
# Epoch 47, Loss: 1.6293572998866482e-05
# Epoch 48, Loss: 1.6292936275886535e-05
# Epoch 49, Loss: 1.6292300053517856e-05

In [ ]:
plot_training_loss(loss_list)

## Model Testing 

In [ ]:
train_acc, test_acc = test(W, b, X_train_df, y_train_df, valid_train, X_test_df, y_test_df, valid_test)
print(f"Final Training Accuracy: {train_acc}\nFinal Testing Accuracy: {test_acc}")

## Saving Model Architecture

In [ ]:
# pickle_filename = 'Results/Model1/Final/model1_optimal.pkl'

In [ ]:
pickle_filename = 'Results/Model1/model1_optimal.pkl'

In [ ]:
model1_optimal = {
    "weights": W,
    "bias": b,
    "loss_list": loss_list,
    "train_acc": train_acc,
    "test_acc": test_acc
}

In [ ]:
with open(pickle_filename, 'wb') as file:
    pickle.dump(model1_optimal, file)

## Loading Model Architecture

In [ ]:
# model1_optimal = {}

In [ ]:
# with open(pickle_filename, 'rb') as file:
#     model1_optimal = pickle.load(file)

In [ ]:
# W = model1_optimal["weights"]
# b = model1_optimal["bias"]

In [ ]:
W, b

## Results

In [ ]:
# loss_list = model1_optimal["loss_list"]
# plot_training_loss(loss_list)

In [ ]:
# train_acc = model1_optimal["train_acc"]
# test_acc = model1_optimal["test_acc"]
# print(f"Final Training Accuracy: {train_acc}\nFinal Testing Accuracy: {test_acc}")

In [ ]:
min_random_acc = 1
max_random_acc = 0

for i in range(100):
    random_W = np.random.randn(23, 5) * 0.01
    random_b = np.zeros((1, 5))
    
    _, random_test_acc = test(random_W, random_b, X_train_df, y_train_df, valid_train, X_test_df, y_test_df, valid_test)
    
    max_random_acc = max(max_random_acc, random_test_acc)
    min_random_acc = min(min_random_acc, random_test_acc)
    
print(f"Random W, b accuracy in range: ({min_random_acc}, {max_random_acc})")

In [ ]:
random_accuracy = {
    "min_random_acc": min_random_acc,
    "max_random_acc": max_random_acc
}

with open('Results/Model1/Final/random_accuracy.pkl', 'wb') as file:
    pickle.dump(random_accuracy, file)

In [ ]:
# random_accuracy = {}

# with open('Results/Model1/Final/random_accuracy.pkl', 'rb') as file:
#     random_accuracy = pickle.load(file)
    
# min_random_acc = random_accuracy["min_random_acc"]
# max_random_acc = random_accuracy["max_random_acc"]

In [ ]:
print(f"Clearly, Model 1 learns and improves in accuracy through training:")
print(f"- Final Test Accuracy ({test_acc}) > Maximum Random Test Accuracy ({max_random_acc})")
print(f"- Loss Decreases from {loss_list[0]} to {loss_list[-1]}\n")
print(f"Model 1 does not overfit:")
print(f"- Final Training Accuracy ({train_acc}) ≈ Final Testing Accuracy ({test_acc})")

## Simulation Testing Setup

In [ ]:
def predict_to_move(bot, prediction):
    next_move = bot
    
    if np.array_equal(prediction, np.array([[1, 0, 0, 0, 0]])):
        next_move = (bot[0], bot[1] + 1)
    elif np.array_equal(prediction, np.array([[0, 1, 0, 0, 0]])):
        next_move = (bot[0], bot[1] - 1)
    elif np.array_equal(prediction, np.array([[0, 0, 1, 0, 0]])):
        next_move = (bot[0] - 1, bot[1])
    elif np.array_equal(prediction, np.array([[0, 0, 0, 1, 0]])):
        next_move = (bot[0] + 1, bot[1])
    else:
        next_move = bot
        
    return next_move

In [ ]:
def predict_with_params(bot, alien_matrix, crew_matrix, d_crew, d_alien, alien_detected, crew_detected):
    alien_probs = determine_probabilities(bot, alien_matrix)
    crew_probs = determine_probabilities(bot, crew_matrix)
    
    X = pd.DataFrame([{
        'bot_x': bot[0],
        'bot_y': bot[1],
            
        'alien_up': alien_probs[0],
        'alien_down': alien_probs[1],
        'alien_left': alien_probs[2],
        'alien_right': alien_probs[3],
        'alien_stay': alien_probs[4],
            
        'crew_up': crew_probs[0],
        'crew_down': crew_probs[1],
        'crew_left': crew_probs[2],
        'crew_right': crew_probs[3],
            
        'd_crew_up': np.float32(d_crew[0]),
        'd_crew_down': np.float32(d_crew[1]),
        'd_crew_left': np.float32(d_crew[2]),
        'd_crew_right': np.float32(d_crew[3]),
        'd_crew_stay': np.float32(d_crew[4]),
        
        'd_alien_up': np.float32(d_alien[0]),
        'd_alien_down': np.float32(d_alien[1]),
        'd_alien_left': np.float32(d_alien[2]),
        'd_alien_right': np.float32(d_alien[3]),
        'd_alien_stay': np.float32(d_alien[4]),
            
        'alien_detected': 1 if alien_detected else 0,
        'crew_detected': 1 if crew_detected else 0,
    }])
    
#     X = pd.DataFrame([{
#         'bot_x': bot[0],
#         'bot_y': bot[1],
            
#         'alien_up': alien_probs[0],
#         'alien_down': alien_probs[1],
#         'alien_left': alien_probs[2],
#         'alien_right': alien_probs[3],
#         'alien_stay': alien_probs[4],
            
#         'crew_up': crew_probs[0],
#         'crew_down': crew_probs[1],
#         'crew_left': crew_probs[2],
#         'crew_right': crew_probs[3],
            
#         'd_crew_up': d_crew[0],
#         'd_crew_down': d_crew[1],
#         'd_crew_left': d_crew[2],
#         'd_crew_right': d_crew[3],
#         'd_crew_stay': d_crew[4],
            
#         'alien_detected': 1 if alien_detected else 0,
#         'crew_detected': 1 if crew_detected else 0,
#     }])
    
#     X = pd.DataFrame([{
#         'bot_x': bot[0],
#         'bot_y': bot[1],

#         'alien_up': alien_probs[0],
#         'alien_down': alien_probs[1],
#         'alien_left': alien_probs[2],
#         'alien_right': alien_probs[3],
#         'alien_stay': alien_probs[4],

#         'crew_up': crew_probs[0],
#         'crew_down': crew_probs[1],
#         'crew_left': crew_probs[2],
#         'crew_right': crew_probs[3],

#         'alien_detected': 1 if alien_detected else 0,
#         'crew_detected': 1 if crew_detected else 0,
#     }], columns=['bot_x', 'bot_y', 'alien_up', 'alien_down', 'alien_left', 'alien_right', 'alien_stay', 'crew_up', 'crew_down', 'crew_left', 'crew_right', 'alien_detected', 'crew_detected'])
    
    
#     X = pd.DataFrame([{
#         'bot_x': bot[0],
#         'bot_y': bot[1],

#         **{'alien_' + str(key): value for key, value in alien_matrix.items()},
#         **{'crew_' + str(key): value for key, value in crew_matrix.items()},

#         'alien_detected': 1 if alien_detected else 0,
#         'crew_detected': 1 if crew_detected else 0
#     }])

#     def filter_features(matrix, feature_prefix):
#         return {feature_prefix + str(key): value for key, value in matrix.items() if feature_prefix + str(key) in high_variance_features}

#     filtered_alien = filter_features(alien_matrix, 'alien_')
#     filtered_crew = filter_features(crew_matrix, 'crew_')

#     X = pd.DataFrame([{
#         'bot_x': bot[0],
#         'bot_y': bot[1],
#         **filtered_alien,
#         **filtered_crew,
#         'alien_detected': 1 if alien_detected else 0,
#         'crew_detected': 1 if crew_detected else 0
#     }])
    
    valid = create_valid_matrix(X)
    
    crew_columns = ['crew_up', 'crew_down', 'crew_left', 'crew_right']
    X['sum_crew'] = X[crew_columns].sum(axis=1)

    for col in crew_columns:
        X.loc[X['sum_crew'] != 0, col] = X[col] / X['sum_crew']

    X.drop('sum_crew', axis=1, inplace=True)
    
    prediction = predict_constrained_stochastic(X, W, b, valid)
    next_move = predict_to_move(bot, prediction)
    
    return next_move

## Bot1 vs. Mimic-Bot1

In [ ]:
def Bot1(k, alpha, max_iter, timeout):
    global grid, open_cells
    
    grid, open_cells = reset_grid(grid, open_cells)
    bot, ship, open_cells = place_bot(grid, open_cells)

    crew_list = []
    alien_list = []
    d_lookup_table = {}

    crew_list, ship = place_crew(ship, open_cells, crew_list)
    alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

    alien_matrix = initialize_alienmatrix(open_cells, bot, k)
    crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)

    win_count = 0
    loss_count = 0
    move = 0
    win_move_count = []
    marker = 0
    
    while (win_count + loss_count) < max_iter:
        neighbors = check_valid_neighbors(len(ship), bot[0], bot[1])
        open_moves = [neigh for neigh in neighbors if (grid[neigh] != 1)]
        open_moves.append(bot)
        next_move = determine_move(open_moves, alien_matrix, crew_matrix) # Determine move deterministically as in Project 2
        
        prev_win_count = win_count
        bot, crew_list, ship, open_cells, win_count, marker = move_bot(ship, bot, next_move, crew_list, alien_list, open_cells, win_count, 1)
        move += 1

        if marker == 1 or move >= timeout:
            loss_count += 1
            print(f"Bot1 captured! Win Count: {win_count}, Loss Count: {loss_count}")

            grid, open_cells = reset_grid(grid, open_cells)
            bot, ship, open_cells = place_bot(grid, open_cells)
            crew_list = []
            alien_list = []
            d_lookup_table = {}

            crew_list, ship = place_crew(ship, open_cells, crew_list)
            alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

            alien_matrix = initialize_alienmatrix(open_cells, bot, k)
            crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
            marker = 0
            move = 0

            continue

        if win_count > prev_win_count:
            print(f"Crew saved! Win Count: {win_count}, Loss Count: {loss_count}")
            win_move_count.append(move)
            move = 0
            d_lookup_table = {}
            alien_matrix = initialize_alienmatrix(open_cells, bot, k)
            crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
        
        print(f"Bot1: {bot}, Crew: {crew_list}, Aliens: {alien_list}")

        alien_matrix, crew_matrix = update_afterbotmove(bot, alien_matrix, crew_matrix)

        # Move bot to optimal neighbor
        marker, alien_list, ship = move_aliens(ship, alien_list, bot) # Move alien randomly

        if marker == 1 or move >= timeout:
            loss_count += 1
            print(f"Bot1 captured! Win Count: {win_count}, Loss Count: {loss_count}")

            grid, open_cells = reset_grid(grid, open_cells)
            bot, ship, open_cells = place_bot(grid, open_cells)
            crew_list = []
            alien_list = []
            d_lookup_table = {}

            crew_list, ship = place_crew(ship, open_cells, crew_list)
            alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

            alien_matrix = initialize_alienmatrix(open_cells, bot, k)
            crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
            marker = 0
            move = 0

            continue
        
        alien_matrix = update_afteralienmove(ship, alien_list, alien_matrix) # Update after alien move
        
        alien_detected = alien_sensor(alien_list, bot, k) # Run Alien Sensor
        crew_detected, d_lookup_table = crew_sensor(ship, bot, alpha, d_lookup_table, crew_list) # Run Crew Sensor
        
        alien_matrix = update_alienmatrix(alien_matrix, alien_detected, bot, k) # Update based on alien sensor

        crew_matrix = update_crewmatrix(crew_matrix, crew_detected, d_lookup_table, bot, alpha) # Update based on crew sensor

    return sum(win_move_count) // max(1, len(win_move_count)), (win_count / max(1, (win_count + loss_count))), win_count

In [ ]:
def Mimic_Bot1(k, alpha, max_iter, timeout):
    global grid, open_cells
    
    grid, open_cells = reset_grid(grid, open_cells)
    bot, ship, open_cells = place_bot(grid, open_cells)

    crew_list = []
    alien_list = []
    d_lookup_table = {}

    crew_list, ship = place_crew(ship, open_cells, crew_list)
    alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

    alien_matrix = initialize_alienmatrix(open_cells, bot, k)
    crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
    
    alien_detected = alien_sensor(alien_list, bot, k) # Initially Run Alien Sensor
    crew_detected, d_lookup_table = crew_sensor(ship, bot, alpha, d_lookup_table, crew_list) # Initially Run Crew Sensor

    win_count = 0
    loss_count = 0
    move = 0
    win_move_count = []
    marker = 0
    
    while (win_count + loss_count) < max_iter:
        neighbors = check_valid_neighbors(len(ship), bot[0], bot[1])
        open_moves = [neigh for neigh in neighbors if (grid[neigh] != 1)]
        open_moves.append(bot)
        
        d_crew, d_lookup_table = determine_d_crew(ship, bot, alpha, d_lookup_table, crew_list, crew_matrix, open_cells)
        d_alien, d_lookup_table = determine_d_alien(ship, bot, alpha, d_lookup_table, alien_list, crew_list, alien_matrix, open_cells)
        
        next_move = predict_with_params(bot, alien_matrix, crew_matrix, d_crew, d_alien, alien_detected, crew_detected) # Predict using optimal W, b for Model 1
        
        prev_win_count = win_count
        bot, crew_list, ship, open_cells, win_count, marker = move_bot(ship, bot, next_move, crew_list, alien_list, open_cells, win_count, 1)
        move += 1

        if marker == 1 or move >= timeout:
            loss_count += 1
            print(f"Mimic-Bot1 captured! Win Count: {win_count}, Loss Count: {loss_count}")

            grid, open_cells = reset_grid(grid, open_cells)
            bot, ship, open_cells = place_bot(grid, open_cells)
            crew_list = []
            alien_list = []
            d_lookup_table = {}

            crew_list, ship = place_crew(ship, open_cells, crew_list)
            alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

            alien_matrix = initialize_alienmatrix(open_cells, bot, k)
            crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
            marker = 0
            move = 0

            continue

        if win_count > prev_win_count:
            print(f"Crew saved! Win Count: {win_count}, Loss Count: {loss_count}")
            win_move_count.append(move)
            move = 0
            d_lookup_table = {}
            alien_matrix = initialize_alienmatrix(open_cells, bot, k)
            crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
        
        print(f"Mimic-Bot1: {bot}, Crew: {crew_list}, Aliens: {alien_list}")

        alien_matrix, crew_matrix = update_afterbotmove(bot, alien_matrix, crew_matrix)

        # Move bot to optimal neighbor
        marker, alien_list, ship = move_aliens(ship, alien_list, bot) # Move alien randomly

        if marker == 1 or move >= timeout:
            loss_count += 1
            print(f"Mimic-Bot1 captured! Win Count: {win_count}, Loss Count: {loss_count}")

            grid, open_cells = reset_grid(grid, open_cells)
            bot, ship, open_cells = place_bot(grid, open_cells)
            crew_list = []
            alien_list = []
            d_lookup_table = {}

            crew_list, ship = place_crew(ship, open_cells, crew_list)
            alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

            alien_matrix = initialize_alienmatrix(open_cells, bot, k)
            crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
            marker = 0
            move = 0

            continue
        
        alien_matrix = update_afteralienmove(ship, alien_list, alien_matrix) # Update after alien move
        
        alien_detected = alien_sensor(alien_list, bot, k) # Run Alien Sensor
        crew_detected, d_lookup_table = crew_sensor(ship, bot, alpha, d_lookup_table, crew_list) # Run Crew Sensor
        
        alien_matrix = update_alienmatrix(alien_matrix, alien_detected, bot, k) # Update based on alien sensor

        crew_matrix = update_crewmatrix(crew_matrix, crew_detected, d_lookup_table, bot, alpha) # Update based on crew sensor

    return sum(win_move_count) // max(1, len(win_move_count)), (win_count / max(1, (win_count + loss_count))), win_count

In [ ]:
def Bot1_vs_MimicBot1(alpha_values, k_values, max_iter, timeout, num_simulations):
    avg_rescue_moves_bot1 = {k: [] for k in k_values}
    prob_crew_rescue_bot1 = {k: [] for k in k_values}
    avg_crew_saved_bot1 = {k: [] for k in k_values}
    
    avg_rescue_moves_mbot1 = {k: [] for k in k_values}
    prob_crew_rescue_mbot1 = {k: [] for k in k_values}
    avg_crew_saved_mbot1 = {k: [] for k in k_values}

    for k in k_values:
        for alpha in alpha_values:
            total_metric1_bot1, total_metric2_bot1, total_metric3_bot1 = 0, 0, 0
            total_metric1_mbot1, total_metric2_mbot1, total_metric3_mbot1 = 0, 0, 0
            
            for i in range(num_simulations):
                metric1_bot1, metric2_bot1, metric3_bot1 = Mimic_Bot1(k, alpha, max_iter, timeout)
                metric1_mbot1, metric2_mbot1, metric3_mbot1 = Mimic_Bot1(k, alpha, max_iter, timeout)
                
                total_metric1_bot1 += metric1_bot1
                total_metric2_bot1 += metric2_bot1
                total_metric3_bot1 += metric3_bot1
                
                total_metric1_mbot1 += metric1_mbot1
                total_metric2_mbot1 += metric2_mbot1
                total_metric3_mbot1 += metric3_mbot1

            avg_metric1_bot1 = total_metric1_bot1 / num_simulations
            avg_metric2_bot1 = total_metric2_bot1 / num_simulations
            avg_metric3_bot1 = total_metric3_bot1 / num_simulations
            
            avg_metric1_mbot1 = total_metric1_mbot1 / num_simulations
            avg_metric2_mbot1 = total_metric2_mbot1 / num_simulations
            avg_metric3_mbot1 = total_metric3_mbot1 / num_simulations

            print(f"Bot1: k={k}, Alpha={alpha}\nAverage Rescue Moves={avg_metric1_bot1}\nProbability of Crew Rescue={avg_metric2_bot1}\nAverage Crew Saved={avg_metric3_bot1}\n")
            print(f"Mimic-Bot1: k={k}, Alpha={alpha}\nAverage Rescue Moves={avg_metric1_mbot1}\nProbability of Crew Rescue={avg_metric2_mbot1}\nAverage Crew Saved={avg_metric3_mbot1}\n")

            avg_rescue_moves_bot1[k].append(avg_metric1_bot1)
            prob_crew_rescue_bot1[k].append(avg_metric2_bot1)
            avg_crew_saved_bot1[k].append(avg_metric3_bot1)
            
            avg_rescue_moves_mbot1[k].append(avg_metric1_mbot1)
            prob_crew_rescue_mbot1[k].append(avg_metric2_mbot1)
            avg_crew_saved_mbot1[k].append(avg_metric3_mbot1)

    return avg_rescue_moves_bot1, prob_crew_rescue_bot1, avg_crew_saved_bot1, avg_rescue_moves_mbot1, prob_crew_rescue_mbot1, avg_crew_saved_mbot1

In [ ]:
def test_simulation_model1(alpha_values, k_values, max_iter, timeout, num_simulations):
    avg_rescue_moves_bot1, prob_crew_rescue_bot1, avg_crew_saved_bot1, avg_rescue_moves_mbot1, prob_crew_rescue_mbot1, avg_crew_saved_mbot1 = Bot1_vs_MimicBot1(alpha_values, k_values, max_iter, timeout, num_simulations)

    prob_crew_rescue_bot1 = {k: [round(prob, 3) for prob in probs] for k, probs in prob_crew_rescue_bot1.items()}
    prob_crew_rescue_mbot1 = {k: [round(prob, 3) for prob in probs] for k, probs in prob_crew_rescue_mbot1.items()}

    print(f"Bot1:\nAverage Rescue Moves = {avg_rescue_moves_bot1}\nProbability of Crew Rescue = {prob_crew_rescue_bot1}\nAverage Crew Saved = {avg_crew_saved_bot1}\n\n")
    print(f"Mimic-Bot1:\nAverage Rescue Moves = {avg_rescue_moves_mbot1}\nProbability of Crew Rescue = {prob_crew_rescue_mbot1}\nAverage Crew Saved = {avg_crew_saved_mbot1}\n")
    
    return avg_rescue_moves_bot1, prob_crew_rescue_bot1, avg_crew_saved_bot1, avg_rescue_moves_mbot1, prob_crew_rescue_mbot1, avg_crew_saved_mbot1

In [ ]:
alpha_values = [0.004]
k_values = [3]
max_iter = 20
timeout = 10000
num_simulations = 20

In [ ]:
metric1_bot1, metric2_bot1, metric3_bot1, metric1_mbot1, metric2_mbot1, metric3_mbot1 = test_simulation_model1(alpha_values, k_values, max_iter, timeout, num_simulations)

In [ ]:
print(metric1_bot1, metric2_bot1, metric3_bot1)
print(metric1_mbot1, metric2_mbot1, metric3_mbot1)

## Obsolete

In [ ]:
# def Bot1(k, alpha, max_iter, timeout):
#     global grid, open_cells
#     bot, ship, open_cells = place_bot(grid, open_cells)

#     crew_list = []
#     alien_list = []
#     d_lookup_table = {}

#     crew_list, ship = place_crew(ship, open_cells, crew_list)
#     alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

#     alien_matrix = initialize_alienmatrix(open_cells, bot, k)
#     crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)

#     win_count = 0
#     loss_count = 0
#     move = 0
#     win_move_count = []
#     marker = 0

#     while (win_count + loss_count) < max_iter:
#         neighbors = check_valid_neighbors(len(ship), bot[0], bot[1])
#         open_moves = [neigh for neigh in neighbors if (grid[neigh] != 1)]
#         open_moves.append(bot) # Bot can stay in place 
#         next_move = determine_move(open_moves, alien_matrix, crew_matrix)
        
#         prev_win_count = win_count
#         bot, crew_list, ship, open_cells, win_count, marker = move_bot(ship, bot, next_move, crew_list, alien_list, open_cells, win_count, 1)
#         move += 1

#         if marker == 1 or move >= timeout:
#             loss_count += 1
#             print(f"Bot captured! Win Count: {win_count}, Loss Count: {loss_count}")

#             grid, open_cells = reset_grid(grid, open_cells)
#             bot, ship, open_cells = place_bot(grid, open_cells)
#             crew_list = []
#             alien_list = []
#             d_lookup_table = {}

#             crew_list, ship = place_crew(ship, open_cells, crew_list)
#             alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

#             alien_matrix = initialize_alienmatrix(open_cells, bot, k)
#             crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
#             marker = 0
#             move = 0

#             continue

#         if win_count > prev_win_count:
#             print(f"Crew saved! Win Count: {win_count}, Loss Count: {loss_count}")
#             win_move_count.append(move)
#             move = 0
#             d_lookup_table = {}
#             alien_matrix = initialize_alienmatrix(open_cells, bot, k)
#             crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
        
#         print(f"Bot: {bot}, Crew: {crew_list}, Aliens: {alien_list}")

#         alien_matrix, crew_matrix = update_afterbotmove(bot, alien_matrix, crew_matrix)

#         # Move bot to optimal neighbor
#         marker, alien_list, ship = move_aliens(ship, alien_list, bot) # Move alien randomly

#         if marker == 1 or move >= timeout:
#             loss_count += 1
#             print(f"Bot captured! Win Count: {win_count}, Loss Count: {loss_count}")

#             grid, open_cells = reset_grid(grid, open_cells)
#             bot, ship, open_cells = place_bot(grid, open_cells)
#             crew_list = []
#             alien_list = []
#             d_lookup_table = {}

#             crew_list, ship = place_crew(ship, open_cells, crew_list)
#             alien_list, ship = place_alien(ship, open_cells, alien_list, bot, k)

#             alien_matrix = initialize_alienmatrix(open_cells, bot, k)
#             crew_matrix = initialize_crewmatrix(open_cells, crew_list, bot)
#             marker = 0
#             move = 0

#             continue
        
#         alien_matrix = update_afteralienmove(ship, alien_list, alien_matrix) # Update after alien move
        
#         alien_detected = alien_sensor(alien_list, bot, k) # Run Alien Sensor
#         crew_detected, d_lookup_table = crew_sensor(ship, bot, alpha, d_lookup_table, crew_list) # Run Crew Sensor
        
#         alien_matrix = update_alienmatrix(alien_matrix, alien_detected, bot, k) # Update based on alien sensor

#         crew_matrix = update_crewmatrix(crew_matrix, crew_detected, d_lookup_table, bot, alpha) # Update based on crew sensor

#     return sum(win_move_count) // max(1, len(win_move_count)), (win_count / max(1, (win_count + loss_count))), win_count

In [ ]:
# def plot_Bot1(alpha_values, k_values, bot1_data, title, metric_num):
#     # Generate a plot for each k-value
#     for k in k_values:
#         plt.figure(figsize=(10, 6))
#         plt.plot(alpha_values, bot1_data[k], label=f'Bot 1, k={k}')
#         plt.title(f'{title} (k={k})')
#         plt.xlabel('alpha')
#         plt.ylabel(title)

#         # Set x-axis ticks
#         plt.xticks(alpha_values, labels=[str(alpha) for alpha in alpha_values])

#         plt.legend()
#         plt.grid(True)
        
#         plt.show()

In [ ]:
# def one_alien_one_crew(alpha_values, k_values, max_iter, timeout, num_simulations):
#     bot1_avg_rescue_moves, bot1_prob_crew_rescue, bot1_avg_crew_saved = Bot1_simulation(alpha_values, k_values, max_iter, timeout, num_simulations)

#     bot1_prob_crew_rescue = {k: [round(prob, 3) for prob in probs] for k, probs in bot1_prob_crew_rescue.items()}

#     print(bot1_avg_rescue_moves, bot1_prob_crew_rescue, bot1_avg_crew_saved, "\n")

#     plot_Bot1(alpha_values, k_values, bot1_avg_rescue_moves, 'Average Rescue Moves', 1)
#     plot_Bot1(alpha_values, k_values, bot1_prob_crew_rescue, 'Probability of Crew Rescue', 2)
#     plot_Bot1(alpha_values, k_values, bot1_avg_crew_saved, 'Average Crew Saved', 3)

In [ ]:
# # Train Function that uses SGD and GD
# def train(X, y, valid, alpha, epochs, initial_batch_size, loss_threshold):
#     num_features = X.shape[1]
#     num_classes = 5
#     n = X.shape[0]

#     W, b = init_params(num_features, num_classes)
# #     print(W, b)

#     previous_loss = float('inf')  # Set starting loss to infinity
#     batch_size = initial_batch_size  # Start with SGD (smaller batch size)
#     switched_to_gd = False
#     loss_list = []  # Store loss values over time
    
#     # Number of times iterated through entire dataset
#     for epoch in range(epochs):
#         current_loss = 0
        
#         # Only iterate over batch size. In SGD, batch size is small, so iterate over smaller batches and update loss
#         for i in range(0, n, batch_size):
#             X_batch = X.iloc[i:i + batch_size]
#             y_batch = y.iloc[i:i + batch_size]
#             valid_batch = valid[i:i + batch_size]

#             y_pred = predict(X_batch, W, b)
# #             print(y_batch, y_pred, valid_batch)
#             batch_loss = loss_function(y_batch, y_pred, valid_batch)
#             current_loss += batch_loss

#             dW, db = compute_gradient(X_batch, y_batch, y_pred, valid_batch)
#             W -= alpha * dW
#             b -= alpha * db

#         current_loss /= (n // batch_size)
#         loss_list.append(current_loss)

#         # Check if loss threshold is met to switch to GD
#         if not switched_to_gd and abs(previous_loss - current_loss) < loss_threshold:
#             batch_size = n  # Set batch size to full dataset (switch to Gradient Descent)
#             switched_to_gd = True
#             print(f"Switched to Gradient Descent. Epoch: {epoch}")

#         previous_loss = current_loss
        
#         print(f"Epoch {epoch}, Loss: {current_loss}")

#     return W, b, loss_list